In [1]:
import os
import sys 
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../../latent_analysis/'))
sys.path.append(os.path.abspath('../../dPCA/python/'))
import LTransform as LT
from dPCA import dPCA
from pathlib import Path
from model import test
from utils import create_directory, load_stuff
import numpy as np
import torch as th
import plot as plot

In [3]:
# Load hidden
data_dir = create_directory(directory_name='loss20')
model_num = 0
model_name = "model{:02d}".format(model_num)

all_hidden = []
ff = [0,8,0,8]

data = []
for i, phase in enumerate(['NF1','FF1','NF2','FF2']):

    weight_file = list(Path(data_dir).glob(f'{model_name}_phase={phase}_*_weights'))[0]
    cfg_file = list(Path(data_dir).glob(f'{model_name}_phase={phase}_*_cfg.json'))[0]

    env, policy, _, _ = load_stuff(cfg_file,weight_file,phase=phase)
    data0, loss, ang_dev, lat_dev = test(env,policy,ff_coefficient=ff[i],is_channel=False)
    data.append(np.array(data0['all_hidden']))

skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for unknown reason
skipping cudagraphs for u

In [5]:
# Apply SVD
n_muscle = 6
n_latent = 3

W = th.load(weight_file)['fc.weight'].numpy()
U, S, Vh = np.linalg.svd(W, full_matrices=True)
V = Vh.T
P = V[:,:n_muscle] # output potent
N = V[:,n_muscle:] # output null

In [8]:
Data_prep =  {'x_pot_ld':[],'x_null_ld':[]}
for i in range(4):
    U=data[i]
    dims=U.shape

    X = U-np.mean(U,axis=0,keepdims=True) # TODO
    X = X.reshape(-1,dims[-1]) # [(cond X time), neuron]
    X = X-np.mean(X,axis=0)

    X_pot = X@(P) # @P.T
    X_null = X@(N) # @N.T

    X_pot = np.reshape(X_pot,newshape=dims[:-1]+(n_muscle,))
    X_null = np.reshape(X_null,newshape=dims[:-1]+(dims[-1]-n_muscle,))
    X = np.reshape(X,newshape=dims)

    if i==0:
        T_p = LT.Transform(num_latent=n_latent)
        T_l = LT.Transform(num_latent=n_latent) 

        T_p.fit(X_pot,method='PCA')
        T_l.fit(X_null,method='PCA')

    x_pot_ld = T_p.transform(X_pot,ensure_orthogonality=True)
    x_null_ld = T_l.transform(X_null,ensure_orthogonality=True)

    Data_prep['x_pot_ld'].append(x_pot_ld)
    Data_prep['x_null_ld'].append(x_null_ld)

In [20]:
%matplotlib qt
fig, ax = plot.plot_traj([Data_prep['x_null_ld'][0], Data_prep['x_null_ld'][1]],which_times=[10],plot_scatter=1,dim=3)